In [ ]:
############ Auteur: Y.S ###########
############ Fev 2024 ##############

# python 3.8.12
# Ce script est fait pour le télécharger les archives GZ des données météo & et les décompresser les fichiers CSV 
# Période 2020-2023 puis 2024-2025 au pas d'une heure 

import os
import requests
import pandas as pd
import numpy as np
import datetime
import time
import sys
import gzip

# ================ Chemins a persopnnaliser =============================
# Chemin d'enregistrement des archives gz et des fichiers CSV décompressés
folder_projet = r"D:\MesDocuments\Formation\DataScientist_PSL\Projet\BD\Meteo" 

folder_gz = os.path.join(folder_projet, 'gz') 

folder_csv= os.path.join(folder_projet, 'CSV')

if not os.path.isdir(folder_gz):
    os.mkdir(folder_gz)
if not os.path.isdir(folder_gz):   
    os.mkdir(folder_csv)

In [ ]:


# ================== Définition des fonctions pour le téléchargement, la décompression et la lecture================================
def convert_to_date(chaine):
    return pd.to_datetime(str(chaine), format='%Y%m%d%H', errors='coerce')

def download_file(url, filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    print('Téléchargement: ', file)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
    else:
        print("Fichier d'archive non présent à l'url habituelle: ", file)

def decompress_gz(filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    
    if os.path.exists(file):
        with gzip.open(file, 'rb') as f_in:
            file= os.path.join(folder_csv, filename) 
            print('Décompression', file)
            with open(file, 'wb') as f_out:
                f_out.write(f_in.read())
    else:
        print("Fichier d'archive non trouvé: ", file)
        print("Téléchargez l'archive GZ, manuellement ou en modifiant la variable 'download', puis relancez le script")

def read_csv(filename):
    file= os.path.join(folder_csv, filename)
    if os.path.exists(file):
        print('Lecture: ', file)
        df= pd.read_csv(file, header=0, sep=";", dtype={"NUM_POSTE":str, 'AAAAMMJJHH':str}, parse_dates=['AAAAMMJJHH'], date_parser= convert_to_date)
    else:
        print("Fichier CSV non trouvé: ", file)
        print("Téléchargez l'archive GZ, manuellement ou en modifiant la variable 'download', puis relancez le script")
    return df


In [ ]:
# ---------------- urls de téléchargement des url_liste_postes   https://meteo.data.gouv.fr/ et des descriptifs des differents champs
# 
url_liste_postes= "https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922"
url_desc_h = "https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_descriptif_champs.csv"

# Lecture en ligne du fichier JSON liste_postes
req = requests.get(url_liste_postes)
if req.status_code == 200:
    data_json= req.json()
else:
    print('la requête a échoué avec le code : ', req.status_codes)
    sys.exit() # interrompt le script
df_liste_postes = pd.DataFrame(data_json['features']).T
# Ajout de colonnes avec les champs de properties
df_liste_postes['lat'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][1]).astype(float)
df_liste_postes['lon'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][0]).astype(float)
df_liste_postes['nom_usuel']= df_liste_postes['properties'].apply(lambda x: x['NOM_USUEL'].strip())
df_liste_postes['num_poste']= df_liste_postes['properties'].apply(lambda x: x['NUM_POSTE'].strip())
df_liste_postes['commune'] = df_liste_postes['properties'].apply(lambda x: x['COMMUNE'].strip())
df_liste_postes['ficheClimComplete'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimComplete']).astype(float)
df_liste_postes['ficheClimReduite'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimReduite']).astype(float)
df_liste_postes['alti'] = df_liste_postes['properties'].apply(lambda x: x['ALTI'])
# supprime les colonnes inutiles et classe par numéro de poste (donc par département)
df_liste_postes.drop(['type', 'geometry', 'properties'], axis=1, inplace=True)
df_liste_postes.sort_values(by= ['num_poste'], inplace=True)


# Lit en ligne le fichier "fin" "H_descriptif_champs.csv" de description des champs
# définit un dataframe pandas avec les 2 colonnes "param" et "name_long" pour la description des champs 
df_desc_f = pd.read_csv(url_desc_h, sep=":", header= None, index_col=0, names= ["param", "name_long", 'complement'], dtype={"param":str, "name_long":str, 'complement':str}, encoding= 'utf-8')
df_desc_f.index= df_desc_f.index.str.strip()
df_desc_f['name_long']= df_desc_f['name_long'].str.strip()

# Enregistrement du dataframe  descriptif
filename = 'H_descriptif_champs_reduit.csv'
file = os.path.join(folder_csv, filename)
liste_a_garder = ['NUM_POSTE', 'NOM_USUEL', 'LAT', 'LON','ALTI', 'AAAAMMJJHH','FF', 'T', 'DIF', 'DIR', 'U']
df_desc_f.loc[liste_a_garder, ['name_long']].to_csv(file)
# ================ Extrait du fichier CSV Descriptif de quelques paramètres les plus souvent utiles (précipitations et températures) =============(les colonnes sont en réalité séparées par ":")
# NUM_POSTE   	 numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL   	 nom usuel du poste
# LAT         	 latitude, négative au sud (en degrés et millionièmes de degré)
# LON         	 longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI        	 altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMMJJHH  	 date de la mesure (année mois jour heure)
# RR1         	 quantité de précipitation tombée en 1 heure (en mm et 1/10 mm)
# DRR1        	 durée des précipitations (en mn)
# FF          	 force du vent moyenné sur 10 mn, mesurée à 10 m (en m/s et 1/10)
# DD          	 direction de FF (en rose de 360)
# FXY         	 valeur maximale de FF dans l’heure (en m/s et 1/10)
# DXY         	 direction de FXY (rose de 360)
# HXY         	 heure de FXY (hhmm)
# FXI         	 force maximale du vent instantané dans l’heure, mesurée à 10 m (en m/s et 1/10)
# DXI         	 direction de FXI (en rose de 360)
# HXI         	 heure de FXI (hhmm)
# FF2         	 force du vent moyenné sur 10 mn, mesurée à 2 m (en m/s et 1/10)
# DD2         	 direction de FF2 (en rose de 360)
# FXI2        	 force maximale du vent instantané dans l’heure, mesurée à 2 m (en m/s et 1/10)
# DXI2        	 direction de FXI2 (en rose de 360)
# HXI2        	 heure de FXI2 (hhmm)
# FXI3S       	 force maximale du vent moyenné sur 3 secondes dans l’heure (en m/s et 1/10)
# DXI3S       	 direction de FXI3S (en rose de 360)
# HXI3S       	 heure de FXI3S (hhmm)
# T           	 température sous abri instantanée (en °C et 1/10)
# TD          	 température du point de rosée (en °C et 1/10)
# TN          	 température minimale sous abri dans l’heure (en °C et 1/10)
# HTN         	 heure de TN (hhmm)
# TX          	 température maximale sous abri dans l’heure (en °C et 1/10)
# HTX         	 heure de TX (hhmm)
# .
# .
# .
# DIF         	  rayonnement diffus horaire en heure UTC (en J/cm2)
# DIR         	  rayonnement direct  horaire en heure UTC (en J/cm2)
# ...
# ======================================================================================
# Enregistrement de la dataframe df_desc_f en "H_descriptif_champs.csv" de description des champs
# Je ne garde que les champs qui nous intéresse :
# ======================================================================================
# NUM_POSTE   	 numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL   	 nom usuel du poste
# LAT         	 latitude, négative au sud (en degrés et millionièmes de degré)
# LON         	 longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI        	 altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMMJJHH  	 date de la mesure (année mois jour heure)
# FF          	 force du vent moyenné sur 10 mn, mesurée à 10 m (en m/s et 1/10)
# T           	 température sous abri instantanée (en °C et 1/10)
# DIF         	  rayonnement diffus horaire en heure UTC (en J/cm2)
# DIR         	  rayonnement direct  horaire en heure UTC (en J/cm2)
# U           	 humidité relative (en %)
# ======================================================================================


# Enregistrement de la dataframe df_liste_postes 
filename = 'liste_postes.csv'
file = os.path.join(folder_csv, filename) 
df_liste_postes.to_csv(file)

In [ ]:
# Les donées méteo 

# ================ Extrait du fichier data CSV ============= (les colonnes sont en réalité séparées par des points-virgules)
# NUM_POSTE	NOM_USUEL	LAT	LON	ALTI	AAAAMMJJHH	RR1	QRR1	DRR1	QDRR1	FF	QFF	DD	QDD	FXY	QFXY
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010100	0.2	1			0	1	0	1	0.8	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010101	0	1			0.5	1	30	1	1.4	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010102	0	1			0.5	1	60	1	0.9	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010103	0	1			0	1	0	1	1	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010104	0	1			0.7	1	160	1	0.7	1



def telechargement(anne_debut):
    print('...Patientez, nous téléchargeons les données horaires à partir de', anne_debut, "...")
    # Structure du nom des fichiers de données QUOTIDIENNES (1 fichier par département dont le numéro 
    # sera ajouté automatiquement au début et à la fin du template ci-dessous)
    if anne_debut == 2024:
        template_start= 'H_' # début du nom du fichier
        template_end='_latest-' + str(anne_debut) + '-' + str(anne_debut+1) + '.csv'
    if anne_debut == 2020:
        template_start= 'H_' # début du nom du fichier
        template_end='_previous-' + str(anne_debut) + '-' + str(anne_debut+3) + '.csv'

    departements = list(np.arange(74,96)) 

    # On affiche la liste
    print('Départements concernés: ', departements)

    # ---------------- urls de téléchargement des archives PAR DEPARTEMENT des dernières données depuis janvier de l'année précédente
    urls= dict()
    for departement in departements:
        if (departement < 10):
            filename = f"{template_start}0{departement}{template_end}" + ".gz" 
        else:
            filename = f"{template_start}{departement}{template_end}" + ".gz" 
        urls[departement]= "https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/" + filename 


    # Téléchargement et décompression des fichiers dans une boucle sur les départements (urls tirées du dictionnaire 'urls')
    download= True
    i, j= 0, 0
    for departement in departements:
        # On récupère l'url
        url = urls[departement]
    # Formation du nom du fichier à partir du template et du numéro de département
        if (departement < 10):
            filename = f"{template_start}0{departement}{template_end}"
        else:
            filename = f"{template_start}{departement}{template_end}"
        if download:
            download_file(url, filename)
            decompress_gz(filename)



# =================== Téléchargement  des données méteo 2020-2023===================
telechargement(2020)
#===================Téléchargement  des données méteo 2024-2025===================
telechargement(2024)       